In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("/content/drive/MyDrive/UC Berkeley Data Analytics Bootcamp/big data/2018.csv")
application_df.tail()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
7213441,2018-12-31,AA,1815,DCA,CLT,1534,1530.0,-4.0,20.0,1550.0,...,100.0,99.0,72.0,331.0,NaN,NaN,NaN,NaN,NaN,NaN
7213442,2018-12-31,AA,1816,CLT,DFW,1751,1757.0,6.0,18.0,1815.0,...,181.0,176.0,148.0,936.0,NaN,NaN,NaN,NaN,NaN,NaN
7213443,2018-12-31,AA,1817,CLT,MEM,2015,2010.0,-5.0,36.0,2046.0,...,112.0,128.0,88.0,511.0,NaN,NaN,NaN,NaN,NaN,NaN
7213444,2018-12-31,AA,1818,CLT,RDU,1300,1323.0,23.0,11.0,1334.0,...,50.0,41.0,26.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN
7213445,2018-12-31,AA,1818,RDU,CLT,1435,1443.0,8.0,8.0,1451.0,...,71.0,59.0,44.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TIME     float64
 19  ACTUAL_ELAPSED_TIME  float64
 20  AIR_TIME             float64
 21  DISTANCE             float64
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

In [4]:
application_df.shape

(7213446, 28)

In [5]:
# Convert flight date to datetime.
application_df["FL_DATE"] = pd.to_datetime(application_df["FL_DATE"])

In [6]:
# Separate the datetime column into columns for year, month, day, and weekday.
application_df["YEAR"]=application_df["FL_DATE"].dt.year
application_df["MONTH"]=application_df["FL_DATE"].dt.month
application_df["DAY"]=application_df["FL_DATE"].dt.day
# application_df["WEEKDAY"]=application_df["FL_DATE"].dt.dayofweek
application_df["WEEKDAY"]=application_df['FL_DATE'].dt.strftime('%A')

In [7]:
# Drop columns that we won't need for our models.
prio_flight_18_df = application_df.drop(["FL_DATE", "OP_CARRIER_FL_NUM", "CANCELLATION_CODE", "WEATHER_DELAY", "LATE_AIRCRAFT_DELAY", "CARRIER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "Unnamed: 27", "CRS_ARR_TIME", "CANCELLED"],axis=1)
prio_flight_18_df.head()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,ARR_DELAY,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,YEAR,MONTH,DAY,WEEKDAY
0,UA,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,-23.0,0.0,268.0,250.0,225.0,1605.0,2018,1,1,Monday
1,UA,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,-24.0,0.0,99.0,83.0,65.0,414.0,2018,1,1,Monday
2,UA,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,-13.0,0.0,134.0,126.0,106.0,846.0,2018,1,1,Monday
3,UA,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,-2.0,0.0,190.0,182.0,157.0,1120.0,2018,1,1,Monday
4,UA,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,14.0,0.0,112.0,106.0,83.0,723.0,2018,1,1,Monday


In [8]:
# Fill all null values with zeros.
prio_flight_18_df = prio_flight_18_df.fillna(0)

In [9]:
# Categorize the scheduled departure time to four key times of day.
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 0) & (prio_flight_18_df["CRS_DEP_TIME"] < 600), "sched_dep_time"] = "Night"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 600) & (prio_flight_18_df["CRS_DEP_TIME"] < 1200), "sched_dep_time"] = "Morning"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 1200) & (prio_flight_18_df["CRS_DEP_TIME"] < 1800), "sched_dep_time"] = "Afternoon"
prio_flight_18_df.loc[(prio_flight_18_df["CRS_DEP_TIME"] >= 1800) & (prio_flight_18_df["CRS_DEP_TIME"] < 2400), "sched_dep_time"] = "Evening"
prio_flight_18_df.head(20)

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,YEAR,MONTH,DAY,WEEKDAY,sched_dep_time
0,UA,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,0.0,268.0,250.0,225.0,1605.0,2018,1,1,Monday,Afternoon
1,UA,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,0.0,99.0,83.0,65.0,414.0,2018,1,1,Monday,Morning
2,UA,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,0.0,134.0,126.0,106.0,846.0,2018,1,1,Monday,Afternoon
3,UA,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,0.0,190.0,182.0,157.0,1120.0,2018,1,1,Monday,Afternoon
4,UA,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,0.0,112.0,106.0,83.0,723.0,2018,1,1,Monday,Morning
5,UA,ORD,OMA,2241,2244.0,3.0,15.0,2259.0,1.0,2.0,...,0.0,93.0,79.0,62.0,416.0,2018,1,1,Monday,Evening
6,UA,IAH,LAS,750,747.0,-3.0,14.0,801.0,854.0,6.0,...,0.0,206.0,193.0,173.0,1222.0,2018,1,1,Monday,Morning
7,UA,DEN,CID,1324,1318.0,-6.0,11.0,1329.0,1554.0,6.0,...,0.0,115.0,102.0,85.0,692.0,2018,1,1,Monday,Afternoon
8,UA,SMF,EWR,2224,2237.0,13.0,10.0,2247.0,627.0,9.0,...,0.0,314.0,299.0,280.0,2500.0,2018,1,1,Monday,Evening
9,UA,RIC,DEN,1601,1559.0,-2.0,12.0,1611.0,1748.0,8.0,...,0.0,252.0,237.0,217.0,1482.0,2018,1,1,Monday,Afternoon


In [10]:
# Categorize the months column by season.
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] == 12) | (prio_flight_18_df["MONTH"] == 1) | (prio_flight_18_df["MONTH"] == 2), "season"] = "Winter"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 2) & (prio_flight_18_df["MONTH"] <= 5), "season"] = "Spring"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 5) & (prio_flight_18_df["MONTH"] <= 8), "season"] = "Summer"
prio_flight_18_df.loc[(prio_flight_18_df["MONTH"] > 8) & (prio_flight_18_df["MONTH"] <= 11), "season"] = "Autumn"

In [11]:
prio_flight_18_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 23 columns):
 #   Column               Dtype  
---  ------               -----  
 0   OP_CARRIER           object 
 1   ORIGIN               object 
 2   DEST                 object 
 3   CRS_DEP_TIME         int64  
 4   DEP_TIME             float64
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  ARR_TIME             float64
 11  ARR_DELAY            float64
 12  DIVERTED             float64
 13  CRS_ELAPSED_TIME     float64
 14  ACTUAL_ELAPSED_TIME  float64
 15  AIR_TIME             float64
 16  DISTANCE             float64
 17  YEAR                 int64  
 18  MONTH                int64  
 19  DAY                  int64  
 20  WEEKDAY              object 
 21  sched_dep_time       object 
 22  season               object 
dtypes: float64(13), int64(4), object

In [12]:
# Categorize each flight as delayed or not, based on if the flight arrived at its destination late.
prio_flight_18_df['DELAY'] = np.where((prio_flight_18_df.ARR_DELAY > 0), 1, 0)
prio_flight_18_df.head()

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,...,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,YEAR,MONTH,DAY,WEEKDAY,sched_dep_time,season,DELAY
0,UA,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,...,250.0,225.0,1605.0,2018,1,1,Monday,Afternoon,Winter,0
1,UA,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,...,83.0,65.0,414.0,2018,1,1,Monday,Morning,Winter,0
2,UA,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,...,126.0,106.0,846.0,2018,1,1,Monday,Afternoon,Winter,0
3,UA,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,...,182.0,157.0,1120.0,2018,1,1,Monday,Afternoon,Winter,0
4,UA,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,...,106.0,83.0,723.0,2018,1,1,Monday,Morning,Winter,1


In [13]:
# Drop remaining delay-related columns and columns that might lead to multi-collinearity:
prio_flight_18_df = prio_flight_18_df.drop(["ARR_DELAY", "DEP_DELAY", "MONTH", "DAY"], axis=1)

In [14]:
# Determine the top 5 airports so we can narrow the scope of the analysis.
prio_flight_18_df["ORIGIN"].value_counts()

ATL    390046
ORD    332953
DFW    279298
DEN    235989
CLT    233317
        ...  
AKN        63
CYS        58
IFP        45
ART        25
YNG         2
Name: ORIGIN, Length: 358, dtype: int64

In [15]:
# Determine the top 5 airports so we can narrow the scope of the analysis.
prio_flight_18_df["DEST"].value_counts()

ATL    390079
ORD    332942
DFW    279272
DEN    236020
CLT    233309
        ...  
AKN        63
CYS        58
IFP        45
ART        26
YNG         2
Name: DEST, Length: 358, dtype: int64

In [16]:
# Determine the top 5 airlines so we can narrow the scope of the analysis.
prio_flight_18_df["OP_CARRIER"].value_counts()

WN    1352552
DL     949283
AA     916818
OO     774137
UA     621565
YX     316090
B6     305010
MQ     296001
OH     278457
9E     245917
AS     245761
YV     215138
EV     202890
NK     176178
F9     120035
G4      96221
HA      83723
VX      17670
Name: OP_CARRIER, dtype: int64

In [17]:
# Create lists to represent the top 5 airports and airlines.
top_airports = ["ATL","ORD","DFW","CLT","DEN"]
top_airlines = ["WN", "DL", "AA", "OO", "UA"]

In [18]:
# Limit the data frame to include only the top 5 airports and airlines.
top_prio_flight_18_df = prio_flight_18_df.loc[prio_flight_18_df['ORIGIN'].isin(top_airports)]
top_prio_flight_18_df = top_prio_flight_18_df.loc[prio_flight_18_df['DEST'].isin(top_airports)]
top_prio_flight_18_df = top_prio_flight_18_df.loc[prio_flight_18_df['OP_CARRIER'].isin(top_airlines)]
top_prio_flight_18_df = top_prio_flight_18_df.reset_index(drop=True)

In [19]:
# Review the columns to determine which need to be scaled and which need one hot encoding.
top_prio_flight_18_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100170 entries, 0 to 100169
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   OP_CARRIER           100170 non-null  object 
 1   ORIGIN               100170 non-null  object 
 2   DEST                 100170 non-null  object 
 3   CRS_DEP_TIME         100170 non-null  int64  
 4   DEP_TIME             100170 non-null  float64
 5   TAXI_OUT             100170 non-null  float64
 6   WHEELS_OFF           100170 non-null  float64
 7   WHEELS_ON            100170 non-null  float64
 8   TAXI_IN              100170 non-null  float64
 9   ARR_TIME             100170 non-null  float64
 10  DIVERTED             100170 non-null  float64
 11  CRS_ELAPSED_TIME     100170 non-null  float64
 12  ACTUAL_ELAPSED_TIME  100170 non-null  float64
 13  AIR_TIME             100170 non-null  float64
 14  DISTANCE             100170 non-null  float64
 15  YEAR             

In [20]:
## Scale the numeric columns:
#flight_data_scaled = StandardScaler().fit_transform(top_prio_flight_18_df[["CRS_DEP_TIME", "DEP_TIME", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "ARR_TIME", "DIVERTED", "CRS_ELAPSED_TIME", "ACTUAL_ELAPSED_TIME", "AIR_TIME", "DISTANCE"]])

In [21]:
## Create a data frame.
#top_prio_flight_scaled = pd.DataFrame(
#    flight_data_scaled,
#    columns=["CRS_DEP_TIME", "DEP_TIME", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "ARR_TIME", "DIVERTED", "CRS_ELAPSED_TIME", "ACTUAL_ELAPSED_TIME", "AIR_TIME", "DISTANCE"]
#)

In [22]:
# Check the data frame.
#top_prio_flight_scaled

In [23]:
# Get dummies for the categorical columns:
flight_dummies = pd.get_dummies(top_prio_flight_18_df[["OP_CARRIER", "ORIGIN", "DEST", "WEEKDAY", "sched_dep_time", "season"]])

In [24]:
# Check the data frame.
flight_dummies

,OP_CARRIER_AA,OP_CARRIER_DL,OP_CARRIER_OO,OP_CARRIER_UA,OP_CARRIER_WN,ORIGIN_ATL,ORIGIN_CLT,ORIGIN_DEN,ORIGIN_DFW,ORIGIN_ORD,...,WEEKDAY_Tuesday,WEEKDAY_Wednesday,sched_dep_time_Afternoon,sched_dep_time_Evening,sched_dep_time_Morning,sched_dep_time_Night,season_Autumn,season_Spring,season_Summer,season_Winter
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100165,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
100166,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
100167,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
100168,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [25]:
# Confirm dummies.
list(flight_dummies.columns)

['OP_CARRIER_AA',
 'OP_CARRIER_DL',
 'OP_CARRIER_OO',
 'OP_CARRIER_UA',
 'OP_CARRIER_WN',
 'ORIGIN_ATL',
 'ORIGIN_CLT',
 'ORIGIN_DEN',
 'ORIGIN_DFW',
 'ORIGIN_ORD',
 'DEST_ATL',
 'DEST_CLT',
 'DEST_DEN',
 'DEST_DFW',
 'DEST_ORD',
 'WEEKDAY_Friday',
 'WEEKDAY_Monday',
 'WEEKDAY_Saturday',
 'WEEKDAY_Sunday',
 'WEEKDAY_Thursday',
 'WEEKDAY_Tuesday',
 'WEEKDAY_Wednesday',
 'sched_dep_time_Afternoon',
 'sched_dep_time_Evening',
 'sched_dep_time_Morning',
 'sched_dep_time_Night',
 'season_Autumn',
 'season_Spring',
 'season_Summer',
 'season_Winter']

In [26]:
# Isolate variables that we're trying to predict.
variables_predicting_df = top_prio_flight_18_df[['DIVERTED','DELAY']]

In [27]:
# Check the data frame.
variables_predicting_df

,DIVERTED,DELAY
0,0.0,0
1,0.0,0
2,0.0,0
3,0.0,0
4,0.0,0
...,...,...
100165,0.0,0
100166,0.0,1
100167,0.0,0
100168,0.0,0


In [28]:
# Concatenate the three data frames.
cleaned_flights_df = pd.concat([top_prio_flight_18_df, flight_dummies], axis=1)

In [29]:
cleaned_flights_df

,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_TIME,...,WEEKDAY_Tuesday,WEEKDAY_Wednesday,sched_dep_time_Afternoon,sched_dep_time_Evening,sched_dep_time_Morning,sched_dep_time_Night,season_Autumn,season_Spring,season_Summer,season_Winter
0,UA,DEN,ORD,915,909.0,12.0,921.0,1216.0,19.0,1235.0,...,0,0,0,0,1,0,0,0,0,1
1,UA,ORD,CLT,2101,2127.0,16.0,2143.0,2359.0,4.0,3.0,...,0,0,0,1,0,0,0,0,0,1
2,UA,ORD,DEN,1345,1342.0,18.0,1400.0,1511.0,8.0,1519.0,...,0,0,1,0,0,0,0,0,0,1
3,UA,ORD,DEN,2245,2246.0,18.0,2304.0,6.0,6.0,12.0,...,0,0,0,1,0,0,0,0,0,1
4,UA,DFW,DEN,915,907.0,13.0,920.0,955.0,16.0,1011.0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100165,AA,DFW,ORD,845,846.0,19.0,905.0,1042.0,7.0,1049.0,...,0,0,0,0,1,0,0,0,0,1
100166,AA,ORD,CLT,1211,1221.0,18.0,1239.0,1508.0,10.0,1518.0,...,0,0,1,0,0,0,0,0,0,1
100167,AA,DFW,CLT,2055,2045.0,20.0,2105.0,2347.0,4.0,2351.0,...,0,0,0,1,0,0,0,0,0,1
100168,AA,CLT,ATL,2100,2100.0,12.0,2112.0,2157.0,10.0,2207.0,...,0,0,0,1,0,0,0,0,0,1


In [30]:
# Finally, need to drop the columns that might interfere with delay predictions:
# Let's check our columns again:
list(cleaned_flights_df.columns)

['OP_CARRIER',
 'ORIGIN',
 'DEST',
 'CRS_DEP_TIME',
 'DEP_TIME',
 'TAXI_OUT',
 'WHEELS_OFF',
 'WHEELS_ON',
 'TAXI_IN',
 'ARR_TIME',
 'DIVERTED',
 'CRS_ELAPSED_TIME',
 'ACTUAL_ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'YEAR',
 'WEEKDAY',
 'sched_dep_time',
 'season',
 'DELAY',
 'OP_CARRIER_AA',
 'OP_CARRIER_DL',
 'OP_CARRIER_OO',
 'OP_CARRIER_UA',
 'OP_CARRIER_WN',
 'ORIGIN_ATL',
 'ORIGIN_CLT',
 'ORIGIN_DEN',
 'ORIGIN_DFW',
 'ORIGIN_ORD',
 'DEST_ATL',
 'DEST_CLT',
 'DEST_DEN',
 'DEST_DFW',
 'DEST_ORD',
 'WEEKDAY_Friday',
 'WEEKDAY_Monday',
 'WEEKDAY_Saturday',
 'WEEKDAY_Sunday',
 'WEEKDAY_Thursday',
 'WEEKDAY_Tuesday',
 'WEEKDAY_Wednesday',
 'sched_dep_time_Afternoon',
 'sched_dep_time_Evening',
 'sched_dep_time_Morning',
 'sched_dep_time_Night',
 'season_Autumn',
 'season_Spring',
 'season_Summer',
 'season_Winter']

In [31]:
# Let's remove these columns (which might impact delay predictions):
# delays_weather_no, delays_weather_yes, delays_late_aircraft_no, delays_late_aircraft_yes, CANCELLED, DIVERTED, DEP_DELAY, ARR_DELAY
# cleaned_flights_df = cleaned_flights_df.drop(["delays_weather_no", "delays_weather_yes", "delays_late_aircraft_no", "delays_late_aircraft_yes", "CANCELLED", "DIVERTED", "DEP_DELAY", "ARR_DELAY"],axis=1)

In [32]:
cleaned_flights_df = cleaned_flights_df.drop(columns=["OP_CARRIER","ORIGIN","DEST","WEEKDAY","sched_dep_time","season"])

In [33]:
cleaned_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100170 entries, 0 to 100169
Data columns (total 44 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CRS_DEP_TIME              100170 non-null  int64  
 1   DEP_TIME                  100170 non-null  float64
 2   TAXI_OUT                  100170 non-null  float64
 3   WHEELS_OFF                100170 non-null  float64
 4   WHEELS_ON                 100170 non-null  float64
 5   TAXI_IN                   100170 non-null  float64
 6   ARR_TIME                  100170 non-null  float64
 7   DIVERTED                  100170 non-null  float64
 8   CRS_ELAPSED_TIME          100170 non-null  float64
 9   ACTUAL_ELAPSED_TIME       100170 non-null  float64
 10  AIR_TIME                  100170 non-null  float64
 11  DISTANCE                  100170 non-null  float64
 12  YEAR                      100170 non-null  int64  
 13  DELAY                     100170 non-null  i

In [34]:
#
X = cleaned_flights_df.drop(columns=['DELAY'])
display(X.head())
y = cleaned_flights_df[['DELAY']]
display(y.head())

,CRS_DEP_TIME,DEP_TIME,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_TIME,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,...,WEEKDAY_Tuesday,WEEKDAY_Wednesday,sched_dep_time_Afternoon,sched_dep_time_Evening,sched_dep_time_Morning,sched_dep_time_Night,season_Autumn,season_Spring,season_Summer,season_Winter
0,915,909.0,12.0,921.0,1216.0,19.0,1235.0,0.0,147.0,146.0,...,0,0,0,0,1,0,0,0,0,1
1,2101,2127.0,16.0,2143.0,2359.0,4.0,3.0,0.0,122.0,96.0,...,0,0,0,1,0,0,0,0,0,1
2,1345,1342.0,18.0,1400.0,1511.0,8.0,1519.0,0.0,162.0,157.0,...,0,0,1,0,0,0,0,0,0,1
3,2245,2246.0,18.0,2304.0,6.0,6.0,12.0,0.0,165.0,146.0,...,0,0,0,1,0,0,0,0,0,1
4,915,907.0,13.0,920.0,955.0,16.0,1011.0,0.0,130.0,124.0,...,0,0,0,0,1,0,0,0,0,1


,DELAY
0,0
1,0
2,0
3,0
4,0


In [35]:
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [36]:
X_test_scaled[0]

array([ 0.09851966,  0.09010266, -0.25516317,  0.14418113,  0.3023872 ,
        1.29862745,  0.32365201, -0.05127495, -0.56986453, -0.62112303,
       -0.83906286, -0.59182577,  0.        ,  1.04497599, -0.60941046,
       -0.22613913, -0.46024291, -0.16886994, -0.54166189, -0.42637963,
       -0.45960571, -0.53298084,  1.86286036, -0.54439504,  2.34317169,
       -0.46136799, -0.52603679, -0.53892747, -0.42163867, -0.4218795 ,
       -0.36992025, -0.40363431,  2.40386768, -0.41114121, -0.41244559,
        1.26694082, -0.48304136, -0.79955583, -0.19595207, -0.56909616,
       -0.58889427, -0.60216351,  1.82069104])

In [37]:
num_of_features = len(X_test_scaled[0])
print(num_of_features)

43


In [41]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = "sigmoid" #hp.Choice('activation',['softmax'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=14,
        max_value=44,
        step=10), activation=activation, input_dim=num_of_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=4,
            max_value=44,
            step=10),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation=activation))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [42]:
# Import the kerastuner library
!pip install keras-tuner --upgrade
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [43]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Trial 56 Complete [00h 01m 23s]
val_accuracy: 0.8750548958778381

Best val_accuracy So Far: 0.8887113928794861
Total elapsed time: 00h 34m 00s


In [56]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(1)
top_hyper[0].values

{'first_units': 44,
 'num_layers': 3,
 'units_0': 4,
 'units_1': 4,
 'units_2': 34,
 'units_3': 14,
 'units_4': 44,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0040'}

In [45]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

783/783 - 1s - loss: 1.4134 - accuracy: 0.4930 - 1s/epoch - 1ms/step
Loss: 1.4134091138839722, Accuracy: 0.49303197860717773
783/783 - 1s - loss: 2.3665 - accuracy: 0.5251 - 787ms/epoch - 1ms/step
Loss: 2.366499423980713, Accuracy: 0.5250968337059021
783/783 - 1s - loss: 0.9922 - accuracy: 0.6528 - 757ms/epoch - 967us/step
Loss: 0.9921832084655762, Accuracy: 0.6527971625328064


# Compiling, Training and Evaluating the Mode

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['first_units'], activation = "sigmoid", input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_1'], activation = "sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_2'], activation = "sigmoid"))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_3'], activation = "sigmoid"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_4'], activation = "sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 44)                1936      
                                                                 
 dense_10 (Dense)            (None, 4)                 180       
                                                                 
 dense_11 (Dense)            (None, 34)                170       
                                                                 
 dense_12 (Dense)            (None, 14)                490       
                                                                 
 dense_13 (Dense)            (None, 44)                660       
                                                                 
 dense_14 (Dense)            (None, 1)                 45        
                                                                 
Total params: 3,481
Trainable params: 3,481
Non-traina

In [49]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [50]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
2348/2348 [==============================] - 4s 1ms/step - loss: 0.4979 - accuracy: 0.7647
Epoch 2/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.3879 - accuracy: 0.8371
Epoch 3/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.3243 - accuracy: 0.8665
Epoch 4/20
2348/2348 [==============================] - 4s 2ms/step - loss: 0.2707 - accuracy: 0.8914
Epoch 5/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.2514 - accuracy: 0.8982
Epoch 6/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.2452 - accuracy: 0.9021
Epoch 7/20
2348/2348 [==============================] - 4s 2ms/step - loss: 0.2416 - accuracy: 0.9030
Epoch 8/20
2348/2348 [==============================] - 5s 2ms/step - loss: 0.2393 - accuracy: 0.9045
Epoch 9/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.2369 - accuracy: 0.9065
Epoch 10/20
2348/2348 [==============================] - 3s 1ms/step - loss: 0.233

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

783/783 - 1s - loss: 0.2233 - accuracy: 0.9046 - 743ms/epoch - 949us/step
Loss: 0.22325089573860168, Accuracy: 0.9045641422271729
